# パケット異常検知

## データの準備

* kddcupのデータをインポートし，それぞれの特徴に対して特徴名を付与する

In [17]:
>>> import pandas
>>> 
>>> col_names = ["duration","protocol_type","service","flag","src_bytes",
...    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
...    "logged_in","num_compromised","root_shell","su_attempted","num_root",
...    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
...    "is_host_login","is_guest_login","count","srv_count","serror_rate",
...    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
...    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
...    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
...    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
...    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
>>> 
>>> kdd_data_10percent = pandas.read_csv("kddcup99/kddcup.data_10_percent", header=None, names = col_names)

* kdd_data_10percentのprotocol_typeを参照

In [14]:
print(kdd_data_10percent.protocol_type)

0         tcp
1         tcp
2         tcp
3         tcp
4         tcp
5         tcp
6         tcp
7         tcp
8         tcp
9         tcp
10        tcp
11        tcp
12        tcp
13        tcp
14        tcp
15        tcp
16        tcp
17        tcp
18        tcp
19        tcp
20        tcp
21        tcp
22        tcp
23        tcp
24        tcp
25        tcp
26        tcp
27        tcp
28        tcp
29        tcp
         ... 
493991    tcp
493992    tcp
493993    tcp
493994    tcp
493995    tcp
493996    tcp
493997    tcp
493998    tcp
493999    tcp
494000    tcp
494001    tcp
494002    tcp
494003    tcp
494004    tcp
494005    tcp
494006    tcp
494007    tcp
494008    tcp
494009    tcp
494010    tcp
494011    tcp
494012    tcp
494013    tcp
494014    tcp
494015    tcp
494016    tcp
494017    tcp
494018    tcp
494019    tcp
494020    tcp
Name: protocol_type, Length: 494021, dtype: object


### データの整形

フィーチャの要素のうち文字列のものにダミー変数を割り振り，数値化する．**(例) http,https=>0,1 など**  
文字列で構成されるフィーチャはprotocol_type，service_list，flag_listの３種であるのが目視で確認できたので，これらに対して適用する．　　

* それぞれのフィーチャの集合を求める => to_set_elems関数
* フィーチャの集合に番号を振る => add_to_index関数
 - この番号がダミー変数となる

In [18]:
protocol_type = kdd_data_10percent['protocol_type']
service = kdd_data_10percent['service']
flag = kdd_data_10percent['flag']

protocol_type_set = service_set = flag_set = []

def to_set_elems(data):
    elems_list = []
    for elem in data:
        elems_list.append(elem)

    return set(elems_list)

def count_set_elems(data):
    return len(to_set_elems(data))

def add_to_index(data):
    temp = []
    for (idx, elem) in enumerate(data):
        temp.append((elem, idx))
    
    return dict(temp)

def convert_to_dummy(data):
    dummy_list = []
    for elem in data[0]:
        for k, v in data[1].items():
            if (k == elem):
                dummy_list.append(v)
    
    return dummy_list

def merge_to_dummy(data):
    return pandas.DataFrame({col:data})

protocol_type_count, service_count, flag_count\
        = map(count_set_elems, (protocol_type, service, flag))
# print(protocol_type_counts, service_counts, flag_counts) # => (3, 66, 11)
protocol_type_set, service_set, flag_set\
        = map(to_set_elems, (protocol_type, service, flag))
# print(protocol_type_set) # => set(['udp', 'icmp', 'tcp'])
protocol_type_index, service_index, flag_index\
        = map(add_to_index, (protocol_type_set, service_set, flag_set))
# print(protocol_type_index, service_index, flag_index)
dummy_protocol_type, dummy_service, dummy_flag\
        = map(convert_to_dummy, ((protocol_type, protocol_type_index),\
                                                           (service, service_index),\
                                                           (flag, flag_index)))

merge_column_list = ['protocol_type', 'service', 'flag']
dummy_data_list = [dummy_protocol_type, dummy_service, dummy_flag]
for (col, data) in zip(merge_column_list, dummy_data_list):
    kdd_data_10percent[col] = merge_to_dummy(data)
print(kdd_data_10percent['protocol_type'])

0         2
1         2
2         2
3         2
4         2
5         2
6         2
7         2
8         2
9         2
10        2
11        2
12        2
13        2
14        2
15        2
16        2
17        2
18        2
19        2
20        2
21        2
22        2
23        2
24        2
25        2
26        2
27        2
28        2
29        2
         ..
493991    2
493992    2
493993    2
493994    2
493995    2
493996    2
493997    2
493998    2
493999    2
494000    2
494001    2
494002    2
494003    2
494004    2
494005    2
494006    2
494007    2
494008    2
494009    2
494010    2
494011    2
494012    2
494013    2
494014    2
494015    2
494016    2
494017    2
494018    2
494019    2
494020    2
Name: protocol_type, Length: 494021, dtype: int64


In [ ]:
# 